In [7]:
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
# import tensorflow as tf

import numpy as np
from matplotlib import pyplot as plt


In [12]:
#load keyframe point clouds
idx1 = 10
fn1 = "keyframe_" + str(idx1) + ".csv"
k1 = np.loadtxt(fn1, delimiter=",")

idx2 = 12
fn2 = "keyframe_" + str(idx2) + ".csv"
k2 = np.loadtxt(fn2, delimiter=",")

In [13]:
plt = Plotter(N = 1, axes = 1, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]          
disp.append(Points(k1, c = 'red', r = 3, alpha = 0.5))
disp.append(Points(k2, c = 'blue', r = 3, alpha = 0.5))
plt.show(disp, "debugging rays_o and rays_d")
# ViewInteractiveWidget(plt.window)  